In [2]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Add bias term to feature matrix
X_train_bias = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
X_test_bias = np.hstack((np.ones((X_test.shape[0], 1)), X_test))

# One-hot encode the target labels
num_classes = len(np.unique(y))
y_train_encoded = np.eye(num_classes)[y_train]

# Initialize parameters
learning_rate = 0.01
num_epochs = 1000

# Initialize weights
num_features = X_train_bias.shape[1]
num_classes = y_train_encoded.shape[1]
weights = np.random.randn(num_features, num_classes)

# Training loop
for epoch in range(num_epochs):
    # Calculate scores
    scores = np.dot(X_train_bias, weights)
    
    # Compute softmax probabilities
    exp_scores = np.exp(scores)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    
    # Compute gradients
    error = probs - y_train_encoded
    gradient = np.dot(X_train_bias.T, error) / X_train_bias.shape[0]
    
    # Update weights
    weights -= learning_rate * gradient

# Make predictions
scores = np.dot(X_test_bias, weights)
predicted_labels = np.argmax(scores, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print("Accuracy:", accuracy)


Accuracy: 0.8888888888888888


In [4]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def gradient_descent(X, y, num_iterations, learning_rate):
    num_samples, num_features = X.shape
    weights = np.zeros((num_features, 1))
    
    for _ in range(num_iterations):
        scores = np.dot(X, weights)
        predictions = sigmoid(scores)
        error = predictions - y
        gradient = np.dot(X.T, error) / num_samples
        weights -= learning_rate * gradient
    
    return weights

def one_vs_rest_train(X_train, y_train, num_classes, num_iterations, learning_rate):
    weights_list = []
    
    for class_label in range(num_classes):
        y_class = (y_train == class_label).astype(int).reshape(-1, 1)
        weights = gradient_descent(X_train, y_class, num_iterations, learning_rate)
        weights_list.append(weights)
    
    return weights_list

def one_vs_rest_predict(X_test, weights_list):
    scores = np.zeros((X_test.shape[0], len(weights_list)))
    
    for idx, weights in enumerate(weights_list):
        scores[:, idx] = np.dot(X_test, weights).flatten()  # Flatten the scores
    predicted_labels = np.argmax(scores, axis=1)
    return predicted_labels

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Add bias term to feature matrix
X_train_bias = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
X_test_bias = np.hstack((np.ones((X_test.shape[0], 1)), X_test))

# Train OvR logistic regression models
num_iterations = 1000
learning_rate = 0.01
num_classes = len(np.unique(y))
weights_list = one_vs_rest_train(X_train_bias, y_train, num_classes, num_iterations, learning_rate)

# Make predictions
predicted_labels = one_vs_rest_predict(X_test_bias, weights_list)

# Calculate accuracy
accuracy = accuracy_score(y_test, predicted_labels)
print("Accuracy:", accuracy)


Accuracy: 0.8666666666666667


In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Load the Iris dataset (3 classes)
iris = load_iris()
X, y = iris.data, iris.target

# Add a bias term to X
X_biased = np.c_[X, np.ones(X.shape[0])]
num_features = X_biased.shape[1]
num_classes = len(np.unique(y))

# Convert target labels to one-hot encoding
y_onehot = np.eye(num_classes)[y]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_biased, y_onehot, test_size=0.2, random_state=42)

# Initialize weights
weights = np.random.randn(num_features, num_classes)

# Softmax activation function
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# Training
learning_rate = 0.01
num_epochs = 1000
for epoch in range(num_epochs):
    # Forward pass
    logits = np.dot(X_train, weights)
    probabilities = softmax(logits)

    # Compute gradients
    error = probabilities - y_train
    gradients = np.dot(X_train.T, error)

    # Update weights
    weights -= learning_rate * gradients

# Testing
logits_test = np.dot(X_test, weights)
probabilities_test = softmax(logits_test)
predicted_classes = np.argmax(probabilities_test, axis=1)

accuracy = np.mean(predicted_classes == np.argmax(y_test, axis=1))
print("Accuracy:", accuracy)


Accuracy: 0.9666666666666667
